In [1]:
# @title [1단계: 설치] (2-in-1 툴)
# (런타임 다시 시작 후 이 셀을 '단 한 번만' 실행하세요)

print(">>> 0. 모든 라이브러리 설치 중...")
# ★ [핵심] OpenPose에 필요했던 'controlnet-aux' 라이브러리를 '제거'했습니다.
!pip install --upgrade transformers diffusers accelerate "pillow<12.0" opencv-python-headless gradio --quiet

print(">>> 설치 완료. [2단계] 셀을 실행하세요.")

>>> 0. 모든 라이브러리 설치 중...
>>> 설치 완료. [2단계] 셀을 실행하세요.


In [8]:
# @title [2단계: 2-in-1 어시스턴트 실행] (Inpaint Strength 추가)

# --- 0. Import ---
import gradio as gr
import torch
import cv2
import numpy as np
from PIL import Image
from diffusers.utils import load_image

from diffusers import (
    StableDiffusionControlNetPipeline,
    StableDiffusionControlNetImg2ImgPipeline,
    StableDiffusionInpaintPipeline,
    ControlNetModel,
    UniPCMultistepScheduler
)

# --- 1. GPU 및 모델 파이프라인 전역 변수 설정 ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

pipe_canny = None
pipe_inpaint_cn = None
pipe_inpaint_vanilla = None

# --- 2. VRAM 관리 및 모델 로드 헬퍼 ---

def clear_vram(except_pipe=None):
    """ VRAM에서 지정된 파이프를 제외하고 모든 AI 모델을 삭제합니다. """
    global pipe_canny, pipe_inpaint_cn, pipe_inpaint_vanilla

    print(f"\nClearing VRAM. Keeping: {except_pipe}")

    if except_pipe != "canny" and pipe_canny is not None:
        pipe_canny.to("cpu"); del pipe_canny; pipe_canny = None

    if except_pipe != "inpaint_cn" and pipe_inpaint_cn is not None:
        pipe_inpaint_cn.to("cpu"); del pipe_inpaint_cn; pipe_inpaint_cn = None

    if except_pipe != "inpaint_vanilla" and pipe_inpaint_vanilla is not None:
        pipe_inpaint_vanilla.to("cpu"); del pipe_inpaint_vanilla; pipe_inpaint_vanilla = None

    torch.cuda.empty_cache()

# --- 3. 백엔드 로직 (Tab 1: Sketch-to-Image) ---

def load_canny_pipe():
    """ 1번 탭(Canny)에 필요한 모델을 VRAM에 로드합니다. """
    global pipe_canny
    if pipe_canny is None:
        clear_vram(except_pipe="canny")
        print("Loading Canny pipeline (ControlNet + SD 1.5)...")
        controlnet = ControlNetModel.from_pretrained(
            "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
        )
        pipe_canny = StableDiffusionControlNetPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            controlnet=controlnet,
            torch_dtype=torch.float16,
            safety_checker=None
        ).to(device)
        pipe_canny.scheduler = UniPCMultistepScheduler.from_config(pipe_canny.scheduler.config)
    print("Canny pipeline is ready.")

def generate_canny(sketch_dict, prompt, neg_prompt, scale, steps):
    if sketch_dict is None: raise gr.Error("스케치 이미지가 필요합니다!")
    sketch_image = sketch_dict["composite"]
    if sketch_image is None: raise gr.Error("스케치 이미지가 비어있습니다!")

    load_canny_pipe()

    image_np = np.array(sketch_image.resize((512, 512)))
    canny_image_np = cv2.Canny(image_np, 100, 200)
    canny_image = Image.fromarray(canny_image_np)

    generator = torch.Generator(device=device).manual_seed(12345)
    result = pipe_canny(
        prompt=prompt,
        negative_prompt=neg_prompt,
        image=canny_image,
        num_inference_steps=steps,
        guidance_scale=scale,
        generator=generator
    ).images[0]

    return result, canny_image

# --- 4. 백엔드 로직 (Tab 2: Partial Edit) ---

def load_inpaint_cn_pipe():
    """ 2번 탭(Inpaint)에 필요한 ControlNet 결합 모델을 VRAM에 로드합니다. """
    global pipe_inpaint_cn
    if pipe_inpaint_cn is None:
        clear_vram(except_pipe="inpaint_cn")
        print("Loading Inpaint-ControlNet pipeline (SD 1.5 + Canny)...")

        controlnet = ControlNetModel.from_pretrained(
            "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
        )
        pipe_inpaint_cn = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            controlnet=controlnet,
            torch_dtype=torch.float16,
            safety_checker=None
        ).to(device)
        pipe_inpaint_cn.scheduler = UniPCMultistepScheduler.from_config(pipe_inpaint_cn.scheduler.config)
    print("Inpaint-ControlNet pipeline is ready.")


def generate_inpaint(image_editor_dict, prompt, neg_prompt, scale, steps, strength): # ★ [수정] strength 파라미터 추가
    if image_editor_dict is None: raise gr.Error("이미지를 업로드하고 수정할 영역을 마스크(칠)해야 합니다!")

    image = image_editor_dict.get("background")
    mask_layers = image_editor_dict.get("layers")

    if image is None: raise gr.Error("원본 이미지가 없습니다!")
    if mask_layers is None or len(mask_layers) == 0 or mask_layers[0] is None:
        raise gr.Error("수정할 영역을 브러시로 칠해야 합니다 (마스크가 없음)!")

    # 1. 이미지 및 마스크 준비 (512x512)
    original_image = image.resize((512, 512)).convert("RGB")
    mask = mask_layers[0].resize((512, 512))
    mask_alpha = mask.split()[-1]

    load_inpaint_cn_pipe() # 2. 모델 로드 (ControlNet 결합 파이프)

    # --- 3. Inpainting을 위한 마스크 처리 ---
    # a. ControlNet이 참고할 '원본 외곽선' 추출 (캐릭터 형태 유지를 위해)
    image_np = np.array(original_image)
    canny_image_np = cv2.Canny(image_np, 100, 200)
    canny_image = Image.fromarray(canny_image_np).convert("RGB")

    # b. 마스크 영역을 '불투명 회색'으로 지우기 (AI가 덮어쓰도록 유도)
    image_to_modify = np.array(original_image)
    mask_alpha_np = np.array(mask_alpha)

    # 마스크가 있는 부분(흰색)을 127 (회색)으로 변경
    image_to_modify[mask_alpha_np > 0] = [127, 127, 127]
    image_to_modify_pil = Image.fromarray(image_to_modify)

    # 4. 생성 (ControlNet Img2Img 파이프라인 사용)
    generator = torch.Generator(device=device).manual_seed(12345)

    # ControlNet Img2Img 파이프라인은 'image', 'control_image'를 받습니다.
    # image = 수정된 이미지, control_image = 원본 Canny 외곽선
    result = pipe_inpaint_cn(
        prompt=prompt + ", detailed, professional", # 프롬프트 디테일 추가
        negative_prompt=neg_prompt + ", worst quality, blurry, extra limbs",
        image=image_to_modify_pil,
        control_image=canny_image,
        strength=strength,                         # ★ [수정] Gradio에서 받은 strength 사용
        num_inference_steps=40,
        guidance_scale=9.0,
        generator=generator,
    ).images[0]

    return result

# --- 5. Gradio UI 실행 ---

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎨 AI 게임 리소스 어시스턴트 (2-in-1)")

    with gr.Tabs():

        # --- 탭 1: Sketch-to-Image ---
        with gr.TabItem("1. Sketch-to-Image (Canny)") as tab1:
            gr.Markdown("**기능:** 스케치를 업로드하고 프롬프트를 입력하면, AI가 스케치에 맞춰 채색/완성합니다.")
            with gr.Row():
                with gr.Column(scale=1):
                    c_sketch = gr.ImageEditor(type="pil", label="스케치 이미지 (Sketch)", value="test_knight.jpg")
                    c_prompt = gr.Textbox(label="긍정 프롬프트", placeholder="a knight in black iron plate armor...")
                    c_neg_prompt = gr.Textbox(label="부정 프롬프트", placeholder="blurry, low quality...")
                    c_scale = gr.Slider(minimum=1, maximum=20, value=7.5, step=0.5, label="Guidance Scale (CFG)")
                    c_steps = gr.Slider(minimum=10, maximum=50, value=30, step=1, label="Steps")
                    c_btn = gr.Button("생성하기 (Generate)", variant="primary")
                with gr.Column(scale=1):
                    c_result = gr.Image(label="결과 이미지 (Result)")
                    c_preview = gr.Image(label="추출된 외곽선 (Canny Preview)")

            gr.Examples(
                examples=[[{"background": "test_knight.jpg", "layers": [], "composite": "test_knight.jpg"}, "a knight in shining armor, fantasy art", "blurry, low quality"]],
                inputs=[c_sketch, c_prompt, c_neg_prompt]
            )

        # --- 탭 2: Partial Edit ---
        with gr.TabItem("2. Partial Edit (Inpainting)") as tab3:
            gr.Markdown("**기능:** 이미지를 업로드하고 **브러시로 수정할 영역을 칠한(Masking)** 뒤, 프롬프트를 입력하면 해당 부분만 수정합니다.")
            with gr.Row():
                with gr.Column(scale=1):
                    i_img = gr.ImageEditor(type="pil", label="수정할 이미지 (Image & Mask)")
                    i_prompt = gr.Textbox(label="긍정 프롬프트 (바꿀 내용)", placeholder="a wooden club (나무 몽둥이)")
                    i_neg_prompt = gr.Textbox(label="부정 프롬프트", placeholder="blurry, low quality, sword...")
                    i_strength = gr.Slider(minimum=0.1, maximum=1.0, value=0.5, step=0.05, label="수정 강도 (Strength) - 낮을수록 원본 이미지 유지") # ★ [수정] 슬라이더 추가
                    i_scale = gr.Slider(minimum=1, maximum=20, value=9.0, step=0.5, label="Guidance Scale (CFG)")
                    i_steps = gr.Slider(minimum=10, maximum=50, value=40, step=1, label="Steps")
                    i_btn = gr.Button("부분 수정 (Inpaint)", variant="primary")
                with gr.Column(scale=1):
                    i_result = gr.Image(label="결과 이미지 (Result)")

    # --- Gradio 버튼과 백엔드 함수 연결 ---
    c_btn.click(fn=generate_canny, inputs=[c_sketch, c_prompt, c_neg_prompt, c_scale, c_steps], outputs=[c_result, c_preview], show_progress='full')
    # ★ [수정] Inpaint 버튼에 'strength' 입력값 추가
    i_btn.click(fn=generate_inpaint, inputs=[i_img, i_prompt, i_neg_prompt, i_scale, i_steps, i_strength], outputs=[i_result], show_progress='full')

print("\nGradio 앱을 실행합니다...")
demo.launch(share=True, debug=True) # Colab에서 외부 접속을 위해 share=True

Using device: cuda

Gradio 앱을 실행합니다...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2d3505baa0baf2e211.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



Clearing VRAM. Keeping: inpaint_cn
Loading Inpaint-ControlNet pipeline (SD 1.5 + Canny)...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Inpaint-ControlNet pipeline is ready.


...

Inpaint-ControlNet pipeline is ready.


  0%|          | 0/20 [00:00<?, ?it/s]

Inpaint-ControlNet pipeline is ready.


  0%|          | 0/20 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2d3505baa0baf2e211.gradio.live
